In [45]:
%pip install llama-index
%pip install llama-index-vector-stores-chroma
%pip install chromadb
%pip install kitchenai-whisk
%pip install kitchenai-llama
%pip install deepeval
%pip install google-genai
%pip install httpx
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Magic Helper

creating a friendly skip magic command for really expensive cells.

In order to use the cell, just remove the %%skip

In [ ]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [48]:
from llama_index.core import VectorStoreIndex, StorageContext, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor, QuestionsAnsweredExtractor
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core import Settings
import chromadb
import logging
import asyncio
import tiktoken
from kitchenai_llama.storage.llama_parser import Parser
import os
import tempfile
from pathlib import Path
from google import genai
from google.genai import types
import pathlib
import asyncio
import nest_asyncio
from deepeval.test_case import LLMTestCase

nest_asyncio.apply()


In [9]:
from google import genai

# Set Your API Key

In [17]:

# Prompt the user to input the Gemini API key
gemini_api_key = input("Please enter your Gemini API key: ")

os.environ["GEMINI_API_KEY"] = gemini_api_key

In [26]:
local_pdf_path = "data/physical-geology-trim100.pdf"  # 618 pages

# Setup logging
logger = logging.getLogger(__name__)

# Initialize token counter
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)
Settings.callback_manager = CallbackManager([token_counter])

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo")
Settings.llm = llm

# Initialize Vector Store
chroma_client = chromadb.PersistentClient(path="chroma_db")
chroma_collection = chroma_client.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Creating a Golen Dataset

We'll use the data from [physcial geology book](./data/physical-geology-trim100.pdf) to test out both of our solutions. To start with, we'll generate a synthetic dataset of inputs, expected outputs, and relevant context. This is called a golden dataset. 

we'll use the golden dataset with both of our solutions to get the performance of our AI 

> **Note:** This is a really expensive operation, so we'll skip it.

In [31]:
%%skip
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.generate_goldens_from_docs(document_paths=['data/physical-geology-trim100.pdf'], max_goldens_per_context=10)

dataset.save_as(
    file_type='json',
    directory="./synthetic_data"
)

Event loop is already running. Applying nest_asyncio patch to allow async execution...


✨ 🧩 ✨ Generating Contexts:  89%|████████▉ | 8/9 [00:02<00:00,  5.39it/s]

Utilizing 8 out of 100 chunks.

✨ 🧩 ✨ Generating Contexts: 100%|██████████| 9/9 [00:04<00:00,  2.12it/s]
✨ Generating up to 30 goldens using DeepEval (using gpt-4o and text-embedding-3-small, method=docs): 100%|██████████| 30/30 [02:36<00:00,  5.22s/it]

Evaluation dataset saved at ./synthetic_data/20250210_104731.json!


'./synthetic_data/20250210_104731.json'

In [ ]:
regular_rag_dataset = EvaluationDataset()
regular_rag_dataset.add_goldens_from_json_file(
    file_path="./synthetic_data/20250210_104731.json", 
    input_key_name="input",
    actual_output_key_name="actual_output",
    expected_output_key_name="expected_output",
    context_key_name="context",
    retrieval_context_key_name="retrieval_context",
)


In [57]:
regular_rag_dataset

EvaluationDataset(test_cases=[], goldens=[Golden(input='Investigate how electron transfer facilitates ion formation in sodium chloride synthesis.', actual_output=None, expected_output='To synthesize sodium chloride (NaCl), electron transfer is essential as it facilitates the formation of ions. Sodium, having 11 electrons, easily gives up its lone third-shell electron, becoming a cation with a positive charge due to an imbalance favoring its protons (11 protons and 10 electrons). Conversely, chlorine, possessing 17 electrons, accepts an additional electron to complete its outer shell, resulting in an anion with a negative charge due to having more electrons (18) than protons (17). The resultant ionic bond between the oppositely charged sodium cation and chlorine anion allows them to stick together, forming NaCl, commonly known as table salt. This electron transfer changes sodium and chlorine atoms into stable ionic forms as part of the bonding process."', context=['Figure 2.2.1 A very s

# Using Gemini 2.0 

now that we have a golden dataset, we can evaluate the performance of the RAG pipeline

In [21]:

client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))


First, upload the pdf to the file api

In [27]:
# Read the PDF file from the local directory
filepath = pathlib.Path(local_pdf_path)

#upload large pdf to file api 
sample_doc = client.files.upload(
  # You can pass a path or a file-like object here
  file=filepath
)

In [28]:
sample_doc

File(name='files/v70thlu1rogb', display_name=None, mime_type='application/pdf', size_bytes=19381269, create_time=datetime.datetime(2025, 2, 10, 16, 38, 48, 989502, tzinfo=TzInfo(UTC)), expiration_time=datetime.datetime(2025, 2, 12, 16, 38, 48, 846150, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 2, 10, 16, 38, 48, 989502, tzinfo=TzInfo(UTC)), sha256_hash='YWUxZjNlNGYxYjI4NTRmZjEyODYwNzNjMWJhZjdhZDRkNDUxNWU3YWE0ZTQ5YTMxZWVkYTYwYzcwNDViNGM1Yg==', uri='https://generativelanguage.googleapis.com/v1beta/files/v70thlu1rogb', download_uri=None, state=<FileState.ACTIVE: 'ACTIVE'>, source=<FileSource.UPLOADED: 'UPLOADED'>, video_metadata=None, error=None)

In [47]:
#simple prompt test
prompt = "Summarize this document"
response = await client.aio.models.generate_content(
  model="gemini-2.0-flash",
  contents=[sample_doc, prompt]
)
print(response.text)

Here's a summary of the document, focusing on the key concepts and takeaways:

**Physical Geology - 2nd Edition Overview**

This textbook provides a comprehensive introduction to physical geology, emphasizing examples from western Canada. It covers fundamental concepts like:

*   **Geology Basics:** Definition of geology as the study of Earth, the importance of geological time, and the scientific methods used in geological inquiry.
*   **Minerals and Rocks:** Definitions of minerals (naturally occurring, specific elemental combinations with repeating 3D structure) and rocks (typically mixtures of minerals). Introduction to rock-forming processes (igneous, sedimentary, metamorphic).
*   **Plate Tectonics:** Fundamentals of Earth's internal structure (core, mantle, crust) and the model of plate tectonics. The role of mantle convection, plate boundaries (divergent, convergent), and processes like subduction.
*   **Geological Time:** The immensity of geological time and the different notat

In [50]:
async def run_test_case(golden, impl_func):
    print(f"Creating test case with golden: {golden.input}")
    # Compute actual output and retrieval context
    actual_output = await impl_func(golden.input)  # Assuming impl_func is async
    retrieval_context = golden.retrieval_context  # Replace with logic to compute retrieval context

    return LLMTestCase(
        input=golden.input,
        actual_output=actual_output,
        retrieval_context=retrieval_context
    )

async def run_all_test_cases(dataset, impl_func):
    tasks = [run_test_case(golden, impl_func) for golden in dataset.goldens]
    test_cases = await asyncio.gather(*tasks)
    for test_case in test_cases:
        dataset.add_test_case(test_case)

async def gemini_impl(input):
    response = await client.aio.models.generate_content(
        model="gemini-2.0-flash",
        contents=[sample_doc, input]
    )
    return response.text

await run_all_test_cases(dataset, gemini_impl)



Creating test case with golden: Investigate how electron transfer facilitates ion formation in sodium chloride synthesis.
Creating test case with golden: Examine how geological methods diverge from chemical and physical sciences in understanding Earth's history.
Creating test case with golden: Compare the reliability of mineral identification using color, streak, and lustre.
Creating test case with golden: By transferring its lone third-shell electron, how does Na form a cation while Cl becomes an anion?
Creating test case with golden: What precise mineral ID roles do color variability, Mohs hardness, and non-metallic lustre play?
Creating test case with golden: Outline geologists' roles in studying Earth's history, natural phenomena, and resource management.
Creating test case with golden: What drives the rock cycle: Earth's internal heat or the sun-powered hydrological cycle?
Creating test case with golden: Elaborate on how Earth's core heat and hydrological cycle sustain an active r

In [62]:
dataset.test_cases[0].actual_output

'Okay, let\'s break down how electron transfer facilitates ion formation in the synthesis of sodium chloride (NaCl), also known as table salt.\n\n**The Fundamental Principle: Achieving Stability**\n\nThe core concept driving the formation of ionic compounds like NaCl is the "octet rule" (though it has exceptions). Atoms tend to gain, lose, or share electrons in order to achieve a stable electron configuration resembling that of noble gases. This means having a full outermost electron shell (typically eight electrons, hence the "octet").\n\n**The Players: Sodium (Na) and Chlorine (Cl)**\n\n1.  **Sodium (Na):**\n\n    *   Sodium is in Group 1 of the periodic table, meaning it has one valence electron (electron in the outermost shell).\n    *   Its electron configuration is 1s² 2s² 2p⁶ 3s¹.\n    *   Losing that single 3s¹ electron will leave it with the configuration 1s² 2s² 2p⁶, which is the same as the noble gas neon (Ne) – a full outermost shell!\n\n2.  **Chlorine (Cl):**\n\n    *   Ch

In [ ]:

# Initialize KitchenAI App
kitchen = KitchenAIApp(namespace="llama-index")


In [ ]:
@kitchen.query.handler("query")
async def query_handler(data: WhiskQuerySchema) -> WhiskQueryBaseResponseSchema:
    """Query handler with RAG"""
    # Create filters from metadata if provided
    filters = None
    if data.metadata:
        from llama_index.core.vector_stores.types import MetadataFilter, MetadataFilters
        print(data.metadata)
        filter_list = [
            MetadataFilter(key=key, value=value)
            for key, value in data.metadata.items()
        ]
        filters = MetadataFilters(filters=filter_list)

    # Create index and query engine
    index = VectorStoreIndex.from_vector_store(vector_store)
    query_engine = index.as_query_engine(
        chat_mode="best",
        filters=filters,
        llm=llm,
        verbose=True
    )

    # Execute query
    response = await query_engine.aquery(data.query)

    # Get token counts
    token_counts = {
        "embedding_tokens": token_counter.total_embedding_token_count,
        "llm_prompt_tokens": token_counter.prompt_llm_token_count,
        "llm_completion_tokens": token_counter.completion_llm_token_count,
        "total_llm_tokens": token_counter.total_llm_token_count
    }
    token_counter.reset_counts()

    return WhiskQueryBaseResponseSchema.from_llama_response(
        data,
        response,
        token_counts=TokenCountSchema(**token_counts),
        metadata={"token_counts": token_counts, **data.metadata} if data.metadata else {"token_counts": token_counts}
    )

In [ ]:

@kitchen.storage.handler("storage")
async def storage_handler(data: WhiskStorageSchema) -> WhiskStorageResponseSchema:
    """Storage handler for document ingestion"""
    try:
        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Use the original filename for the temporary file
            temp_file_path = Path(temp_dir) / Path(data.name).name
            
            # Write bytes data to temporary file
            with open(temp_file_path, 'wb') as f:
                f.write(data.data)
            
            # Initialize parser and load the file
            parser = Parser(api_key=os.environ.get("LLAMA_CLOUD_API_KEY", None))
            response = parser.load(str(temp_dir), metadata=data.metadata)
            
            # Setup storage context and process documents
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            
            # Create index with transformations
            VectorStoreIndex.from_documents(
                response["documents"],
                storage_context=storage_context,
                transformations=[
                    TokenTextSplitter(),
                    TitleExtractor(),
                    QuestionsAnsweredExtractor()
                ],
                show_progress=True
            )
            

            return WhiskStorageResponseSchema(
                id=data.id,
                name=data.name,
                label=data.label,
                data=data.data,
            )
            
    except Exception as e:
        logger.error(f"Error in storage handler: {str(e)}")
        raise